# Data Retrieval

In [1]:
import pandas as pd
import csv
import re
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor

In [2]:
# File path for cities: San Francisco(sf), 
# Okaland(ok), Santa Clara(sc), Santa Cruze(scz)

file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}/calendar.csv'

sf_file = file_url.format('sf_airbnb')
ok_file = file_url.format('ok_airbnb')
sc_file = file_url.format('sc_airbnb')
scz_file = file_url.format('scz_airbnb')

In [3]:
# Loaded price by date information for each city

pricing_sf = pd.read_csv(sf_file, parse_dates=['date'])
pricing_ok = pd.read_csv(ok_file, parse_dates=['date'])
pricing_sc = pd.read_csv(sc_file, parse_dates=['date'])
pricing_scz = pd.read_csv(scz_file, parse_dates=['date'])

print(pricing_sf.shape, pricing_ok.shape, pricing_sc.shape, pricing_scz.shape)

(2420680, 4) (1057770, 4) (2068820, 4) (573050, 4)


In [4]:
# Concatenate four files into one
# Created keys for each file for future references

frames = [pricing_sf, pricing_ok, pricing_sc, pricing_scz]
keys = ['pricing_sf', 'pricing_ok', 'pricing_sc', 'pricing_scz']

df_pricing = pd.concat(frames, keys=keys)

df_pricing.head(10)

listing_id       date available price
pricing_sf 0         958 2019-08-05         f   NaN
           1         958 2019-08-04         f   NaN
           2         958 2019-08-03         f   NaN
           3         958 2019-08-02         f   NaN
           4         958 2019-08-01         f   NaN
           5         958 2019-07-31         f   NaN
           6         958 2019-07-30         f   NaN
           7         958 2019-07-29         f   NaN
           8         958 2019-07-28         f   NaN
           9         958 2019-07-27         f   NaN

# Data Exploration

## Filter down to data with pricing information

In [5]:
# Filter to houses that are available and have listing prices

df_pricing_smaller = df_pricing[df_pricing['available']=='t']
df_pricing_smaller = df_pricing_smaller.copy()

print('Dimension for df_pricing_smaller: ', df_pricing_smaller.shape)
print ('\nNumber of Unique ids: ', df_pricing_smaller.listing_id.nunique())

Dimension for df_pricing_smaller:  (2853357, 4)

Number of Unique ids:  14770


In [6]:
df_pricing_smaller.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [7]:
display( df_pricing_smaller.date.describe(include='all'))

count                 2853357
unique                    389
top       2018-11-03 00:00:00
freq                    10393
first     2018-08-06 00:00:00
last      2019-08-29 00:00:00
Name: date, dtype: object

## Format Conversion and Outlier Removal

In [8]:
# Noted currencies are formated as strings and need to convert to float

def currency2float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [9]:
# Converted price to float format

df_pricing_smaller['price'] = df_pricing_smaller['price'].apply(
                              lambda x: currency2float(x))

In [10]:
# Removed outliers based on 'price'. Use 5% and 95% percentiles as cutoff

five_percentile = np.percentile(df_pricing_smaller.price, 5)
ninety_five_percentile = np.percentile(df_pricing_smaller.price, 95)

df_pricing_smaller = df_pricing_smaller[(
                         df_pricing_smaller.price>=five_percentile)
                        & (df_pricing_smaller.price<=ninety_five_percentile)]

print ('df_pricing_smaller dimensions: ', df_pricing_smaller.shape)

df_pricing_smaller dimensions:  (2573002, 4)


## Filter Data

In [11]:
# Added 'month' column to extract of month of date
# Added 'day_of_week' column to extact day of week. 
# Default format: Monday=0, ..., Sunday=6

df_pricing_smaller['month'] = df_pricing_smaller['date'].dt.month
df_pricing_smaller['day_of_week'] = df_pricing_smaller['date'].dt.weekday


print('df_pricing_smaller dimensions', df_pricing_smaller.shape)
display(df_pricing_smaller.head())

df_pricing_smaller dimensions (2573002, 6)


listing_id       date available  price  month  day_of_week
pricing_sf 63         958 2019-05-02         t  181.0      5            3
           64         958 2019-05-01         t  181.0      5            2
           65         958 2019-04-30         t  181.0      4            1
           66         958 2019-04-29         t  181.0      4            0
           67         958 2019-04-28         t  181.0      4            6

In [12]:
df_pricing_smaller.isnull().sum()

listing_id     0
date           0
available      0
price          0
month          0
day_of_week    0
dtype: int64

In [ ]:
display(df_pricing_smaller.describe(include ='all'))

In [ ]:
df_by_month_day = df_pricing_smaller.groupby(['listing_id', 'month', 'day_of_week'])  \
                                    .price.mean()  \
                                    .reset_index()
df_by_month_day.head()

In [ ]:
df_month_day_counts = df_by_month_day.groupby(['listing_id'])  \
                                     .price.count()  \
                                 .reset_index()
df_month_day_counts.rename(columns = {'price':'counts'}, inplace=True)
df_month_day_counts.head()

In [ ]:
# Find out listings that have full price information (12 months and 7 days/week)

listing_with_12m_7d_price = list(df_month_day_counts[
                                df_month_day_counts.counts==12*7]
                                .listing_id)

In [ ]:
df_pricing_available = df_pricing_smaller[df_pricing_smaller.listing_id.isin(
                                          listing_with_12m_7d_price)]
df_pricing_available.shape

In [ ]:
df_pricing_available.describe(include='all')

In [ ]:
# Group by month by weekday, and calculate an average pricing

df_pricing_by_month_weekday = df_pricing_available.groupby(  \
                              ['listing_id', 'month', 'day_of_week']) \
                              .price.mean()  \
                              .reset_index()
df_pricing_by_month_weekday.head()

In [ ]:
df_pricing_by_month_weekday.describe(include='all')

In [ ]:
# Check for null values if any

df_pricing_by_month_weekday.isnull().sum()

In [ ]:
# Rename the average price column to avoid confusion

df_pricing_by_month_weekday.rename(columns={'price':'price_by_month_week'}, inplace=True)

In [ ]:
df_pricing_by_month_weekday.shape

In [ ]:
df_pricing_by_month_weekday.head()

In [13]:
df_pricing_by_month_weekday = df_pricing_smaller

In [14]:
def get_month(number):
    """
    Given a number, return the name of month.
    
    """
    month_lookup = {1:'Jan.', 2:'Feb.', 3:'Mar.', 4:'Apr.', 5:'May.', 6:'Jun.',
                    7:'Jul.', 8:'Aug.', 9:'Sep.', 10:'Oct.', 11:'Nov.', 12:'Dec.'}
    
    return month_lookup[number]

In [15]:
def get_weekdays(number):
    """
    Given a number, return the name of month.
    
    """
    weekday_lookup = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 
                      4:'Friday', 5:'Saturday', 6:'Sunday'}
    
    return weekday_lookup[number]

In [16]:
df_pricing_by_month_weekday['month'] = df_pricing_by_month_weekday['month']  \
                                       .apply(lambda x: get_month(x))

In [17]:
df_pricing_by_month_weekday['day_of_week'] = df_pricing_by_month_weekday['day_of_week']  \
                                             .apply(lambda x: get_weekdays(x))

In [18]:
df_pricing_final = pd.get_dummies(df_pricing_by_month_weekday, 
                                  prefix='',
                                  prefix_sep='',
                                  columns=['month', 'day_of_week'])

In [19]:
df_pricing_final.columns

Index(['listing_id', 'date', 'available', 'price', 'Apr.', 'Aug.', 'Dec.',
       'Feb.', 'Jan.', 'Jul.', 'Jun.', 'Mar.', 'May.', 'Nov.', 'Oct.', 'Sep.',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'],
      dtype='object')

In [20]:
df_pricing_final.shape

(2573002, 23)

In [21]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'

pkl_path = pkl_file.format('listings_with_selected_features')

df_listings_reviews = pd.read_pickle(pkl_path)
df_listings_reviews.head()

id  bedrooms  cleaning_fee  room_type_Entire home/apt  reviews_per_month  \
0   958       1.0       2.00000                          1               1.38   
1  5858       2.0       2.00000                          1               0.99   
2  7918       1.0       1.69897                          0               0.16   
3  8142       1.0       1.69897                          0               0.15   
4  8567       2.0       2.09691                          1               0.27   

   city_San Francisco  city_Oakland  bathrooms  accommodates  \
0                   1             0        1.0             3   
1                   1             0        1.0             5   
2                   1             0        4.0             2   
3                   1             0        4.0             2   
4                   1             0        1.0             6   

   number_of_reviews        ...         neighbourhood_cleansed_Palo Alto  \
0                152        ...                                        0   
1                112        ...                                        0   
2                 17        ...                                        0   
3                  7        ...                                        0   
4                 30        ...                                        0   

   host_is_superhost_f  amenities_Lock on bedroom door  \
0                    0                               0   
1                    1                               0   
2                    1                               1   
3                    1                               1   
4                    1                               0   

   amenities_Private entrance  amenities_Familykid friendly  \
0                           1                             1   
1                           1                             1   
2                           1                             0   
3                           1                             1   
4                           0                             1   

   district_Santa Clara  neighbourhood_cleansed_San Jose  \
0                     0                                0   
1                     0                                0   
2                     0                                0   
3                     0                                0   
4                     0                                0   

   host_identity_verified_f   ref_district  ref_jurisdiction  
0                         0  San Francisco     SAN FRANCISCO  
1                         0  San Francisco     SAN FRANCISCO  
2                         0  San Francisco     SAN FRANCISCO  
3                         0  San Francisco     SAN FRANCISCO  
4                         1  San Francisco     SAN FRANCISCO  

[5 rows x 33 columns]

In [22]:
df_listings_reviews.shape

(10352, 33)

In [23]:
df_final = pd.merge(left=df_pricing_final, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [24]:
df_final.shape

(1721991, 56)

In [25]:
df_final.isnull().sum()

listing_id                                  0
date                                        0
available                                   0
price                                       0
Apr.                                        0
Aug.                                        0
Dec.                                        0
Feb.                                        0
Jan.                                        0
Jul.                                        0
Jun.                                        0
Mar.                                        0
May.                                        0
Nov.                                        0
Oct.                                        0
Sep.                                        0
Friday                                      0
Monday                                      0
Saturday                                    0
Sunday                                      0
Thursday                                    0
Tuesday                           

In [26]:
df_final.shape

(1721991, 56)

In [27]:
df_final.columns

Index(['listing_id', 'date', 'available', 'price', 'Apr.', 'Aug.', 'Dec.',
       'Feb.', 'Jan.', 'Jul.', 'Jun.', 'Mar.', 'May.', 'Nov.', 'Oct.', 'Sep.',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday', 'id', 'bedrooms', 'cleaning_fee',
       'room_type_Entire home/apt', 'reviews_per_month', 'city_San Francisco',
       'city_Oakland', 'bathrooms', 'accommodates', 'number_of_reviews',
       'host_listings_count', 'review_scores_rating_log10', 'host_yrs',
       'guests_included', 'calculated_host_listings_count',
       'review_scores_location_log10', 'beds', 'yrs_since_first_review',
       'host_response_rate', 'amenities_Air conditioning',
       'review_scores_value_log10', 'jurisdiction_names_Santa Cruz County, CA',
       'amenities_Cable TV', 'neighbourhood_cleansed_Palo Alto',
       'host_is_superhost_f', 'amenities_Lock on bedroom door',
       'amenities_Private entrance', 'amenities_Familykid friendly',
       'district_Santa Cla

In [28]:
# Drop duplicate columns

df_final.drop(['id'], axis=1, inplace=True)

In [29]:
df_final.shape

(1721991, 55)

In [30]:
# Create correlation matrix
corr_matrix = df_final.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

to_drop

['review_scores_value_log10']

In [31]:
# Drop features that have high correlation

df_final.drop(to_drop, axis=1, inplace=True)

In [32]:
df_final.shape

(1721991, 54)

In [ ]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'
pkl_path = pkl_file.format('listings_seasonality')

with open(pkl_path, 'wb') as file:
        pd.to_pickle(df_final, file)

In [ ]:
file_updated = True
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'
pkl_path = pkl_file.format('listings_seasonality')


if file_updated:
    with open(pkl_path, 'wb') as file:
        pd.to_pickle(df_final, file)
else:
    try: 
        with open(pkl_path, 'rb') as file:
            df_final = pd.read_pickle(file)
    except:
        with open(pkl_path, 'wb') as file:
            pd.to_pickle(df_final, file)

In [42]:
# Perform multiple modeling for price prediction

X = df_final.drop(['price', 'date', 'available','ref_district', 'ref_jurisdiction'], axis=1)
y = df_final.loc[:, ['listing_id', 'price']]

In [43]:
grp_by_listing_ids = df_final.groupby('listing_id').first().reset_index().listing_id

In [44]:
listing_ids = list (grp_by_listing_ids)

In [45]:
num_of_grps = len(grp_by_listing_ids)

training_ids = np.random.choice(listing_ids, int(num_of_grps*0.75), replace=False)
print(len(training_ids))
training_ids

7097


array([16327454, 13031597, 24202800, ..., 12432125, 17509000, 18614096])

In [46]:
testing_ids = set(grp_by_listing_ids) - set (training_ids)
len(testing_ids)

2366

In [47]:
X_train = X[X['listing_id'].isin(training_ids)].drop('listing_id', axis=1)
X_test = X[X['listing_id'].isin(testing_ids)].drop('listing_id', axis=1)
y_train = y[y['listing_id'].isin(training_ids)].drop('listing_id', axis=1).values.ravel()
y_test = y[y['listing_id'].isin(testing_ids)].drop('listing_id', axis=1).values.ravel()

print(len(X_train), len(X_test), len(y_train), len(y_test))

1288054 433937 1288054 433937


In [48]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [49]:
# Model-LR

model=sm.OLS(y_train, sm.add_constant(X_train_trans))
fit=model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                 3.999e+04
Date:                Fri, 15 Feb 2019   Prob (F-statistic):               0.00
Time:                        23:52:06   Log-Likelihood:            -7.1903e+06
No. Observations:             1288054   AIC:                         1.438e+07
Df Residuals:                 1288005   BIC:                         1.438e+07
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        159.6386      0.057   2817.985      0.000     159.528     159.750
x1          -4.29e+09   2.89e+10     -0.149      0.882   -6.09e+10    5.23e+10
x2         -3.583e+09   2.41e+10     -0.149      0.882   -5.08e+10    4.37e+10
x3         -4.716e+09   3.17e+10     -0.149      0.882   -6.69e+10    5.75e+10
x4           -4.3e+09   2.89e+10     -0.149      0.882    -6.1e+10    5.24e+10
x5         -4.865e+09   3.27e+10     -0.149      0.882    -6.9e+10    5.93e+10
x6         -4.191e+09   2.82e+10     -0.149      0.882   -5.94e+10    5.11e+10
x7           -4.1e+09   2.76e+10     -0.149      0.882   -5.82e+10       5e+10
x8         -4.345e+09   2.92e+10     -0.149      0.882   -6.16e+10    5.29e+10
x9         -4.235e+09   2.85e+10     -0.149      0.882   -6.01e+10    5.16e+10
x10        -4.679e+09   3.15e+10     -0.149      0.882   -6.64e+10     5.7e+10
x11        -4.685e+09   3.15e+10     -0.149      0.882   -6.65e+10    5.71e+10
x12        -4.029e+09   2.71e+10     -0.149      0.882   -5.72e+10    4.91e+10
x13        -9.125e+10   3.83e+11     -0.238      0.812   -8.42e+11     6.6e+11
x14        -9.236e+10   3.88e+11     -0.238      0.812   -8.52e+11    6.68e+11
x15        -9.109e+10   3.82e+11     -0.238      0.812   -8.41e+11    6.59e+11
x16        -9.181e+10   3.85e+11     -0.238      0.812   -8.47e+11    6.64e+11
x17        -9.175e+10   3.85e+11     -0.238      0.812   -8.47e+11    6.63e+11
x18         -9.22e+10   3.87e+11     -0.238      0.812   -8.51e+11    6.67e+11
x19        -9.197e+10   3.86e+11     -0.238      0.812   -8.49e+11    6.65e+11
x20           24.9587      0.097    258.098      0.000      24.769      25.148
x21            8.7204      0.070    124.730      0.000       8.583       8.857
x22           23.8970      0.078    306.560      0.000      23.744      24.050
x23           -6.5713      0.095    -69.185      0.000      -6.757      -6.385
x24           22.5663      0.348     64.881      0.000      21.885      23.248
x25           -1.7973      0.250     -7.196      0.000      -2.287      -1.308
x26            3.5621      0.067     53.322      0.000       3.431       3.693
x27           30.3882      0.125    242.165      0.000      30.142      30.634
x28           -2.3398      0.102    -22.830      0.000      -2.541      -2.139
x29            5.4296      0.079     68.746      0.000       5.275       5.584
x30           -4.0693      0.114    -35.713      0.000      -4.293      -3.846
x31           -0.2759      0.073     -3.760      0.000      -0.420      -0.132
x32            1.8303      0.072     25.567      0.000       1.690       1.971
x33           -8.2803      0.081   -102.356      0.000      -8.439      -8.122
x34            7.3543      0.115     64.224      0.000       7.130       7.579
x35           -3.6480      0.112    -32.544      0.000      -3.868      -3.428
x3

In [ ]:
# Model-Random Forest

rf = RandomForestRegressor(n_estimators=50, 
                           criterion='mse', 
                           random_state=42, 
                           n_jobs=-1)
rf.fit(X_train_trans, y_train.ravel())

y_train_pred = rf.predict(X_train_trans)
y_test_pred = rf.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

In [ ]:
RMSE = 1043.202**0.5
RMSE

In [ ]:
df_final.listing_id.nunique()

In [ ]:
# Double check on price range. The range is as expected

y.describe(include='all')

In [ ]:
# Map feature importance with labels

feature_labels = X.columns
feature_tuples = zip (feature_labels, rf.feature_importances_)
feature_lists = [list(feature_tuple) for feature_tuple in feature_tuples]

In [ ]:
# Order features based on importance

feature_lists.sort(key=lambda x: abs(x[1]), reverse=True)
feature_lists

In [ ]:
features = [list_[0] for list_ in feature_lists]
features

In [ ]:
columns =['listing_id']
columns.extend(features)
columns.extend(['ref_district', 'ref_jurisdiction', 'price_by_month_week'])

In [ ]:
df_data = df_final[columns]
df_data.head()

In [ ]:
df_data.shape

In [ ]:
df_data.rename(columns={'price_by_month_week':'price'}, inplace=True)

In [ ]:
df_data.isnull().sum()

In [ ]:
temp_df = df_data[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].copy()

days = temp_df.apply(lambda x: x.idxmax(), axis=1)
days

In [ ]:
csv_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

csv_path = csv_file.format('airbnb_data')

df_data.to_csv(csv_path)

In [ ]:
# This is to create visual to show seasonality

df_visual = pd.merge(left=df_pricing_by_month_weekday, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [ ]:
df_visual.shape

In [ ]:
df_visual.columns

In [ ]:
df_visual_entire_apartment = df_visual[df_visual['room_type_Entire home/apt']==1 ]
df_visual_single_room = df_visual[df_visual['room_type_Entire home/apt']!=1 ]

print(df_visual_entire_apartment.shape, df_visual_single_room.shape )

In [ ]:
df_visual_entire_apartment.ref_district.describe

In [ ]:
df_visual_entire_apartment_sf = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='San Francisco']
df_visual_entire_apartment_scz = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='Santa Clara']

print(df_visual_entire_apartment_sf.shape, df_visual_entire_apartment_scz.shape )

In [ ]:
df_visual_entire_apartment_sf.id.nunique()

In [ ]:
df_visual_entire_apartment_scz.id.nunique()

In [ ]:
# Weekly pattern for entire apartment

df_week_entire_apartment = df_visual_entire_apartment.groupby(['ref_district', 'day_of_week'])  \
                                                     .price_by_month_week.mean()
df_week_entire_apartment

In [ ]:
# Weekly pattern for single room

df_week_single_room = df_visual_single_room.groupby(['ref_district', 'day_of_week'])  \
                                           .price_by_month_week.mean()
df_week_single_room

In [ ]:
# Montly Pattern for entire apartment

df_visual_entire_apartment.groupby(['ref_district', 'month']).price_by_month_week.mean()

In [ ]:
# Montly Pattern for single room

df_visual_single_room.groupby(['ref_district', 'month']).price_by_month_week.mean()